# Subword Tokenizer
- 특정 도메인에 특화된 데이터 셋팅 가능

In [1]:
# 네이버 영화 리뷰 데이터
import urllib.request
import os

def get_file(filename, origin):
    cache_dir = os.path.expanduser('~/.torch/datasets')
    os.makedirs(cache_dir,exist_ok=True)
    filepath = os.path.join(cache_dir, filename)

    if not os.path.exists(filepath):
        print(f'Downloading data from {origin}')
        urllib.request.urlretrieve(origin,filepath)

    return filepath

ratings_train_path = get_file("ratings_train.txt", "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt")
ratings_test_path = get_file("ratings_test.txt", "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt")
ratings_train_path,ratings_test_path

('C:\\Users\\ljh10/.torch/datasets\\ratings_train.txt',
 'C:\\Users\\ljh10/.torch/datasets\\ratings_test.txt')

In [ ]:
import pandas as pd

ratings_train_df = pd.read_csv(ratings_train_path, sep='\t')
ratings_test_df = pd.read_csv(ratings_test_path, sep='\t')

display(ratings_train_df)
display(ratings_test_df)

In [3]:
# 결측치 제거
ratings_train_df = ratings_train_df.dropna(how='any')
ratings_test_df = ratings_test_df.dropna(how='any') # how의 인자로 any를 넘기면 행 열 아무거나 비어있어도 드랍함

ratings_train_df.shape, ratings_test_df.shape

((149995, 3), (49997, 3))

In [4]:
# txt 팡리 생성 - 학습 데이터
with open('naver_review.txt', 'w', encoding='utf-8') as f:
    for doc in ratings_train_df['document'].values:
        f.write(doc+'\n')

### SentencePieceTokenizer

In [ ]:
# !pip install sentencepiece

   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   --------------------------------------- 992.0/992.0 kB 11.8 MB/s eta 0:00:00


In [6]:
import sentencepiece as spm

input = 'naver_review.txt'
vocab_size = 10000 
model_prefix = 'naver_review'
cmd = f'--input={input} --model_prefix={model_prefix} --vocab_size={vocab_size}'

spm.SentencePieceTrainer.Train(cmd)


In [ ]:
sp = spm.SentencePieceProcessor()
sp.Load(f'{model_prefix}.model')

for doc in ratings_train_df['document'].values[:3]:
    print(doc)
    print(sp.encode_as_pieces(doc))
    print(sp.encode_as_ids(doc))
    print()


아 더빙.. 진짜 짜증나네요 목소리
['▁아', '▁더빙', '..', '▁진짜', '▁짜증나', '네요', '▁목소리']
[62, 877, 5, 31, 2019, 68, 1710]

흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
['▁흠', '...', '포스터', '보고', '▁초딩', '영화', '줄', '....', '오', '버', '연기', '조차', '▁가볍지', '▁않', '구나']
[1634, 8, 4908, 159, 1460, 33, 264, 60, 173, 548, 410, 1224, 7396, 754, 440]

너무재밓었다그래서보는것을추천한다
['▁너무', '재', '밓', '었다', '그래서', '보', '는것을', '추천', '한다']
[23, 369, 9781, 429, 3780, 143, 6266, 1945, 314]



In [ ]:
text = ratings_test_df['document'][100]
tokens = sp.encode_as_pieces(text)
id_tokens = sp.encode_as_ids(text)
print(text)
print(tokens)
print(id_tokens)

print(''.join(tokens).replace('▁', " ").strip())

print(sp.decode_pieces(tokens))
print(sp.decode_ids(id_tokens))


걸작은 몇안되고 졸작들만 넘쳐난다.
['▁걸작', '은', '▁몇', '안되고', '▁졸작', '들만', '▁넘', '쳐', '난다', '.']
[1060, 18, 621, 6979, 728, 3291, 165, 705, 1003, 4]
걸작은 몇안되고 졸작들만 넘쳐난다.
걸작은 몇안되고 졸작들만 넘쳐난다.
걸작은 몇안되고 졸작들만 넘쳐난다.


### BertWordPieceTokenizer

In [4]:
from tokenizers import BertWordPieceTokenizer

# lower_case : 대소문자 구분 인자
# strip_accents : 악센트 제거 인자
tokenizer = BertWordPieceTokenizer(lowercase=False, strip_accents=False)
vocab_size = 10000

tokenizer.train(
    files=['naver_review.txt'],
    vocab_size = vocab_size, # vocab 사이즈
    min_frequency = 5, # 최소 빈도수 2번 이상 나온 단어만 사용
    show_progress=True # 학습 진행 상황을 보여줌
)


In [ ]:
encoded = tokenizer.encode('아버지가 방에 들어가신다.') # 토큰화된 결과를 반환
print(encoded.tokens) # 토큰화된 결과
print(encoded.ids)  # 토큰에 대응하는 인덱스

['아버지가', '방', '##에', '들어가', '##신', '##다', '.']
[8317, 482, 1032, 6765, 1215, 1027, 16]


In [ ]:
decoded = tokenizer.decode(encoded.ids) # 토큰화된 인덱스를 텍스트로 반환
decoded

'아버지가 방에 들어가신다.'